In [51]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

from collections import defaultdict
import pandas as pd
import numpy as np

# Load data


In [52]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')

In [53]:
bill_df = pd.read_sql_query('select * from "bill_api"',con=engine)
sponsor_df = pd.read_sql_query('select * from "sponsor_api"',con=engine)

In [54]:
bill_df.head()

,biennium,bill_id,class,description,htm_create_date,htm_last_modified_date,htm_url,long_friendly_name,name,type
0,1991-92,HB 1001,Bills,None,1991-08-30T00:00:00,2006-07-10T17:13:53.543,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1001,1001,House Bills
1,1991-92,SHB 1001,Bills,None,1991-02-01T00:00:00,2006-07-10T17:13:54.903,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1001,1001-S,House Bills
2,1991-92,HB 1002,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.637,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1002,1002,House Bills
3,1991-92,HB 1003,Bills,None,1991-01-14T00:00:00,2006-07-10T17:13:11.747,http://app.leg.wa.gov/documents/billdocs/1991-...,House Bill 1003,1003,House Bills
4,1991-92,SHB 1003,Bills,None,1991-02-21T00:00:00,2006-07-10T17:14:07.357,http://app.leg.wa.gov/documents/billdocs/1991-...,Substitute House Bill 1003,1003-S,House Bills


In [72]:
sponsor_df

,sponsor_agency,sponsor_first_name,sponsor_id,sponsor_last_name,sponsor_long_name,sponsor_name,sponsor_order,sponsor_type,biennium,bill_id
0,House,Gary,251,Locke,Representative Locke,Locke,1,Primary,1991-92,HB 1001
1,House,Clyde,17,Ballard,Representative Ballard,Ballard,2,Secondary,1991-92,HB 1001
2,House,Marlin,11,Appelwick,Representative Appelwick,Appelwick,3,Secondary,1991-92,HB 1001
3,House,Wilson,328,Peery,Representative Peery,Peery,4,Secondary,1991-92,HB 1001
4,House,Curtis,254,Ludwig,Representative Ludwig,Ludwig,5,Secondary,1991-92,HB 1001
5,House,Jennifer,34,Belcher,Representative Belcher,Belcher,6,Secondary,1991-92,HB 1001
6,House,Eugene,339,Prince,Representative Prince,Prince,7,Secondary,1991-92,HB 1001
7,House,Holly,301,Myers,Representative H. Myers,H. Myers,8,Secondary,1991-92,HB 1001
8,House,Randy,429,Tate,Representative Tate,Tate,9,Secondary,1991-92,HB 1001
9,House,Christopher,444,Vance,Representative Vance,Vance,10,Secondary,1991-92,HB 1001


In [100]:
def create_secondary_sponsor_column(sponsor_df):
    '''Create a column named secondary_sponsors that contains a list of secondary sponsors'''
    # Create dictionaryn with bill_id and biennium as keys and list of secondary sponsors as values
    s = defaultdict(list)
    for s_id, s_type, biennium, bill_id in zip(sponsor_df['sponsor_id'], 
                                               sponsor_df['sponsor_type'],
                                               sponsor_df['biennium'], 
                                               sponsor_df['bill_id']):
        if s_type == 'Secondary':
            s[(bill_id, biennium)].append(s_id)

    # Create a matrix that can be turned in to a dataframe. Column one is bill_id, column two is 
    # biennium, column three is the list of secondary sponsors
    sponsor_matrix = []
    for k, v in s.items():
        row = []
        row.append(k[0])
        row.append(k[1])
        row.append(v)
        sponsor_matrix.append(row)
        
    sponsors_in_lists_df = pd.DataFrame(sponsor_matrix)
    sponsors_in_lists_df.columns = ['bill_id', 'biennium', 'secondary_sponsors']
        
    sponsor_df_reformatted = sponsor_df[sponsor_df['sponsor_type'] == 'Primary']
    sponsor_df_reformatted = sponsor_df_reformatted.rename(index=str, columns={"sponsor_id": "primary_sponsor_id"})
    sponsor_df_reformatted = sponsor_df_reformatted.drop(['sponsor_type', 'sponsor_order', 'sponsor_last_name', 
                                 'sponsor_long_name', 'sponsor_first_name', 'sponsor_name'], axis = 1)
    
    return sponsor_df_reformatted.merge(sponsors_in_lists_df, how='outer', on=['bill_id', 'biennium'])

In [101]:
sponsor_df_reformatted = create_secondary_sponsor_column(sponsor_df)

In [102]:
sponsor_df_reformatted

,sponsor_agency,primary_sponsor_id,biennium,bill_id,secondary_sponsors
0,House,251,1991-92,HB 1001,"[17, 11, 328, 254, 34, 339, 301, 429, 444, 406..."
1,House,251,1991-92,HB 1002,"[180, 394, 23, 285, 332, 207, 484, 110, 474, 4..."
2,House,311,1991-92,HB 1003,"[110, 474, 54]"
3,House,311,1991-92,HB 1004,"[110, 475, 32, 54, 304]"
4,House,311,1991-92,HB 1005,"[23, 475]"
5,House,311,1991-92,HB 1006,[110]
6,House,311,1991-92,HB 1007,"[180, 285, 110, 475, 443, 276, 48, 472]"
7,House,311,1991-92,HB 1008,"[110, 474, 475]"
8,House,174,1991-92,HB 1009,"[472, 484, 478, 341, 475, 443, 54, 286, 481, 349]"
9,House,174,1991-92,HB 1010,"[472, 409, 294, 86, 484, 475, 320, 481]"
